# Set up environment

In [1]:
%matplotlib inline

import anndata as ad
import ast
import copy
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
import geopandas as gpd
import gseapy
import hdf5plugin
import importlib
import itertools
from math import sqrt
import matplotlib as mpl
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import ScalarFormatter
from matplotlib.colors import LinearSegmentedColormap
from multiprocessing import Pool
import numpy as np
from numpy.polynomial import Polynomial
import os
import pandas as pd
import pickle as pk
import random
import scanpy as sc
import scipy
from scipy import spatial
from scipy.spatial import cKDTree
from scipy.spatial import distance
from scipy.stats import fisher_exact, ks_2samp
from sklearn.cluster import KMeans
import seaborn as sns
from shapely.geometry import Point
import skbio
import slide_tcr_functions
import squidpy as sq
from statistics import mean, stdev
import statsmodels.stats
from statsmodels.stats import multitest
from statsmodels.nonparametric.smoothers_lowess import lowess
import sys
import time
from tqdm.notebook import tqdm
from scipy.interpolate import make_interp_spline
from collections import defaultdict
from numpy.polynomial.polynomial import Polynomial

from alphashape import alphashape
from collections import Counter
from functools import partial
from libpysal.weights import KNN
from matplotlib.collections import PatchCollection
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from perm_test_functions import perm_test_to_final_plot
from plotting_functions import gaussian_heat_plot, plot_cortex_medulla
from shapely.geometry import Point
from slide_tcr_functions import (
    PuckReplicate,
    save_puck,
    load_puck,
    make_puck,
    make_adata,
    load_adata,
    pkl_load,
    pkl_dump
)
from statsmodels import stats
from tcr_mapping import tcr_mapping, get_barcode_position, build_6mer_dist, barcode_matching

# Set directory paths
directory = '../data/'  # Directory with all data
output_directory = '../results/'

# Set plotting parameters
fontprops = fm.FontProperties(size=18)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams['ps.fonttype'] = 42

pixels_needed = 500 / 0.65

master_df = pd.read_csv(f'{directory}Sample_information.csv', header=0)

# Initialize dictionaries and lists
csv_name_to_puck_name = {}
puck_name_to_timepoint = {}
age_to_ss = {}
puck_to_time = {}
csv_names = []

# Process rows in the master dataframe
for index, row in master_df.iterrows():
    puck_name = row['NovaSeq alignment directory']
    age = row['Sample Age']

    # Skip rows with specified Barcodes to TCR csv values
    csv_name = f'{puck_name[11:25]}_B.csv'

    # Populate dictionaries and lists
    csv_name_to_puck_name[csv_name] = puck_name
    csv_names.append(csv_name)
    puck_name_to_timepoint[puck_name] = age

    if age not in age_to_ss:
        age_to_ss[age] = []
    age_to_ss[age].append(row['NovaSeq alignment directory'])
    puck_to_time[puck_name] = age

# Define mappings and orderings
time_to_puck = {puck_to_time[i]: i for i in puck_to_time}

# Define timepoints, day mappings, and puck names

tpt_to_pn = {
    i: j
    for i, j in zip(master_df['Sample Age'],
                    master_df['NovaSeq alignment directory'])
}
tpt_order = [
    '0 day', '1 day', '2 day', '3 day', '4 day', '5 day', '7 day', '10 day',
    '13 day', '3wks', '4 wks', '5 wks', '7wks', '9 wks', '13 wks', '15wks',
    '28wks', '32wks', '48 wks', '68 wks', '90wks'
]

timepoints = [
    '1 day', '2 day', '3 day', '4 day', '5 day', '7 day', '10 day', '13 day',
    '3wks', '4 wks', '5 wks', '7wks', '9 wks', '13 wks', '15wks', '28wks',
    '32wks', '48 wks', '68 wks', '90wks'
]
tpt_to_day = {
    '0 day': 0,
    '1 day': 1,
    '2 day': 2,
    '3 day': 3,
    '4 day': 4,
    '5 day': 5,
    '7 day': 7,
    '10 day': 10,
    '13 day': 13,
    '3wks': 3 * 7,
    '4 wks': 4 * 7,
    '5 wks': 5 * 7,
    '7wks': 7 * 7,
    '9 wks': 9 * 7,
    '13 wks': 13 * 7,
    '15wks': 15 * 7,
    '28wks': 28 * 7,
    '32wks': 32 * 7,
    '48 wks': 48 * 7,
    '68 wks': 68 * 7,
    '90wks': 90 * 7
}

puck_names = ["2023-01-24_Puck_221024_23", # 0 day
                      "2022-11-23_Puck_221024_24", # 0 day
                "2022-11-23_Puck_221024_31", # 1 day
                      "2023-01-16_Puck_221024_27", # 1 day
                      "2022-11-23_Puck_221024_34", # 2 day
                      "2023-01-24_Puck_221024_33", # 2 day
                      "2023-01-24_Puck_221024_36", # 2 day
                      "2022-11-23_Puck_221024_38", # 3 day
                      "2023-01-24_Puck_221024_37", # 3 day
                      "2023-01-24_Puck_221024_39", # 3 day
                      "2023-01-24_Puck_221103_09", # 4 day
                      "2023-01-24_Puck_221103_12", # 4 day
                      "2023-01-24_Puck_221019_08", # 5 day
                      "2023-01-24_Puck_221019_05", # 5 day
                      "2023-01-24_Puck_221019_10", # 7 day
                      "2023-01-24_Puck_221019_14", # 7 day
                      "2022-12-16_Puck_221006_22_trunc", # 10 day
                      "2023-01-24_Puck_221006_23", # 10 day
                      "2022-12-16_Puck_221011_32_trunc", # 13 day
                      "2022-12-16_Puck_221011_33_trunc", # 13 day
                      "2023-01-24_Puck_221118_38", # 3 weeks
                      "2023-01-24_Puck_221118_39", # 3 weeks
                      "2023-01-24_Puck_221103_22", # 4 weeks
                      "2023-01-24_Puck_221118_40", # 4 weeks
                      "2023-01-24_Puck_220930_18", # 5 weeks
                      "2023-01-24_Puck_220930_20", # 5 weeks
                      "2023-01-24_Puck_220930_22", # 5 weeks
                      "2023-01-24_Puck_220930_23", # 7 weeks
                      "2023-01-24_Puck_220930_28", # 7 weeks
                      "2023-01-24_Puck_221011_37", # 9 weeks
                      "2023-01-24_Puck_221011_39", # 9 weeks
                      "2023-01-24_Puck_221011_12", # 13 weeks
                      "2023-01-24_Puck_221011_13", # 13 weeks
                      "2022-11-04_Puck_221006_24", # 15 week
                      "2022-12-16_Puck_221006_26_trunc", # 15 week
                      "2022-11-04_Puck_221006_29", # 28 week
                      "2022-12-16_Puck_221006_35_trunc", # 28 week
                      "2022-11-04_Puck_220930_32", # 32 week
                      "2022-12-16_Puck_220930_29_trunc", # 32 week
                      "2023-01-24_Puck_220930_35", # 32 week
                      "2023-01-24_Puck_221014_37", # 48 week
                      "2023-01-24_Puck_221019_03", # 48 week
                      "2022-12-16_Puck_221011_19_trunc", # 68 week
                      "2022-12-16_Puck_221011_20_trunc", # 68 week
                      "2023-01-24_Puck_221006_32", # 68 week
                      "2022-12-16_Puck_220930_37_trunc", # 90 week
                      "2022-12-16_Puck_220930_38_trunc"] # 90 week

puck_times = [0,0,1,1,2,2,2,3,3,3,4,4,5,5,7,7,10,10,13,13,3*7,3*7,4*7,4*7,5*7,5*7,5*7,
              7*7,7*7,9*7,9*7,13*7,13*7,15*7,15*7,28*7,28*7,32*7,32*7,32*7,48*7,48*7,68*7,68*7,68*7,90*7,90*7]


puck_name_to_days = dict(zip(puck_names, puck_times))

# Define batch mappings
batch_to_tpt = dict(zip(range(len(puck_times)), puck_times))
batch_to_puck_name = dict(zip(range(len(puck_times)), puck_names))
puck_to_batch = dict(zip(puck_names, range(len(puck_times))))

def calculate_morans_i(x,y,label,label_of_interest):
    # Create a GeoDataFrame
    geometry = [Point(xy) for xy in zip(x, y)]
    gdf = gpd.GeoDataFrame({'x': x, 'y': y, 'label': label}, geometry = geometry)

    # Create spatial weights matrix using K-nearest neighbors
    k = 5  # You can adjust the number of neighbors (k) as needed
    w = KNN.from_dataframe(gdf, k=k)

    # Calculate Moran's I for subset
    moran_ct1 = Moran(gdf['label'].apply(lambda x: 1 if x == label_of_interest else 0), w)

    # Print the Moran's I statistics
    print(f"Moran's I for {label_of_interest} Points:")
    print("I:", moran_ct1.I)
    print("Expected I:", moran_ct1.EI)
    print("p-value:", moran_ct1.p_sim)

    return moran_ct1.I

def find_indices(lst, target_item):
    return [index for index, item in enumerate(lst) if item == target_item]

def dilate_cortex_points(bc_cortex, bc_medulla, bc_loc_dict_s1,
                         loc_to_bc_s1, matched_bead_barcodes,
                         loc_to_diversity, title, filt_plot=False,
                         show_plot=True):
    """
    Dilate cortex points based on the median diversity of nearby points.

    Parameters:
    - bc_cortex: List of cortex barcodes
    - bc_medulla: List of medulla barcodes
    - bc_loc_dict_s1: Dictionary mapping barcodes to locations
    - loc_to_bc_s1: Dictionary mapping locations to barcodes
    - matched_bead_barcodes: List of matched bead barcodes
    - locs_cortex: List of cortex locations
    - filt_plot: Boolean flag to filter the plot

    Returns:
    - Dictionary mapping cortex locations to diluted diversity values
    """

    # Extract cortex locations
    cortex_locs = [bc_loc_dict_s1[bc] for bc in bc_cortex]

    # Extract all locations (cortex + medulla)
    everywhere_locs = [bc_loc_dict_s1[bc] for bc in bc_medulla + bc_cortex]

    # If there are fewer than 2 cortex locations, skip dilation
    if len(cortex_locs) < 2:
        return 'Skip'

    point_tree = cKDTree(cortex_locs)

    # Create a copy of loc_to_diversity for cortex locations
    cort_loc_to_diversity = copy.deepcopy(loc_to_diversity)

    # Iterative dilation process
    num_iter = 0
    while num_iter != 2:  # Limiting the number of iterations for demonstration (can be adjusted)
        num_iter += 1
        # print(f"Iteration: {num_iter}")
        # print(f"Number of Cortex Locations: {len(cort_loc_to_diversity)}")

        # Find unassigned cortex locations
        unassigned_cortex_locs = [loc for loc in cortex_locs if loc not in cort_loc_to_diversity]

        # Dilate unassigned cortex locations
        for loc in unassigned_cortex_locs:
            # Find points within a radius of 20 units
            idx_closeby_points = point_tree.query_ball_point(loc, r=20 / 0.65)
            nearest_points = [cortex_locs[idx] for idx in idx_closeby_points]

            # Extract diversity values for nearby points
            div = [cort_loc_to_diversity[i] for i in nearest_points if i in cort_loc_to_diversity]

            # If there are diversity values, assign the median to the current location
            if len(div) > 0:
                cort_loc_to_diversity[loc] = np.median(div)

        # Make a plot
        if (show_plot & num_iter==2):
            print('success')
            plot_dilated_cortex(cort_loc_to_diversity, everywhere_locs, cortex_locs, title, filt_plot)
    return cort_loc_to_diversity


def plot_dilated_cortex(cort_loc_to_diversity, everywhere_locs, cortex_locs, title, filt_plot, vmin=3.3, vmax=5.4):
    """
    Plot the dilated cortex.

    Parameters:
    - cort_loc_to_diversity: Dictionary mapping cortex locations to diluted diversity values
    - everywhere_locs: List of all locations (cortex + medulla)
    - cortex_locs: List of cortex locations
    - locs_cortex: List of cortex locations for filtering
    - filt_plot: Boolean flag to filter the plot
    """
    fig, ax = plt.subplots(figsize=(5, 5))

    # Plot everywhere locations in gray
    x2, y2 = zip(*everywhere_locs)
    plt.scatter(x2, y2, c='gray', s=1, alpha=0.05)  # Everywhere

    # Plot cortex locations in gray with higher alpha
    x2, y2 = zip(*cortex_locs)
    plt.scatter(x2, y2, c='gray', s=1, alpha=0.2)

    # Plot dilated cortex locations
    if filt_plot:
        locs_filt, c = zip(*[(loc, c) for loc, c in zip(list(cort_loc_to_diversity.keys()), list(cort_loc_to_diversity.values())) if loc in cortex_locs])
        x, y = zip(*locs_filt)
    else:
        x, y = zip(*list(cort_loc_to_diversity.keys()))
        c = list(cort_loc_to_diversity.values())

    # Scatter plot with color map
    p = plt.scatter(x, y, c=c, s=1, cmap='inferno', vmin=vmin, vmax=vmax)

    # Additional plot settings
    plt.axis('off')
    plt.xlim([0, 5000])
    plt.ylim([0, 5000])
    plt.title(title)
    cb_ax = fig.add_axes([.91, .124, .04, .754])
    fig.colorbar(p, orientation='vertical', cax=cb_ax)
    fontprops = fm.FontProperties(size=18)
    scalebar = AnchoredSizeBar(
        ax.transData,
        pixels_needed,
        "",
        "lower right",
        pad=0.1,
        color="black",
        frameon=False,
        size_vertical=1,
        fontproperties=fontprops,
    )
    ax.add_artist(scalebar)

    plt.show()

def plot_gene_expression_on_pucks(adata_all, pucks_to_plot, genes_to_plot,title_name = None):
    """
    Plot gene expression on specified pucks for selected genes.

    Parameters:
    - pucks_to_plot: List of puck names to be plotted
    - genes_to_plot: List of gene names to be plotted

    Returns:
    - None
    """
    vmax_calc = {i: [] for i in genes_to_plot}
    vmax_for_genes = {i: 0 for i in genes_to_plot}  # identify 90th percentile
    vmin_calc = {i: [] for i in genes_to_plot}
    vmin_for_genes = {i: 0 for i in genes_to_plot}  # identify 90th percentile
    all_x_agg = {}
    all_y_agg = {}
    all_adata_filt = {}
    all_total_counts = {}

    # Calculate vmax values
    for pn in tqdm(pucks_to_plot):
        if pn not in loaded_pucks:
            adata = adata_all[adata_all.obs.puck_name == pn]
        else:
            adata = loaded_pucks[pn]
        all_x = adata.obs['x_val']
        all_y = adata.obs['y_val']

        all_x_agg[pn] = all_x
        all_y_agg[pn] = all_y

        adata = adata[adata.obs['cortex_medulla'].isin(['cortex', 'medulla'])]  # clean up areas outside tissue
        all_adata_filt[pn] = adata

        total_counts_per_cell = np.sum(adata.X, axis=1)
        all_total_counts[pn] = total_counts_per_cell

        for gene_name in genes_to_plot:
            gene_idx = list(adata.var.index).index(gene_name)
            gene_filt = adata.X[:, gene_idx]
            gene_filt = [i / x * 10000 for i, x in zip(gene_filt, total_counts_per_cell)]

            bc_indices = [index for index, val in enumerate(gene_filt) if val > 0]
            gene_filt = [val for index, val in enumerate(gene_filt) if val > 0]

            vmax_calc[gene_name] = vmax_calc[gene_name] + gene_filt
            vmin_calc[gene_name] = vmin_calc[gene_name] + gene_filt

    # Set percentile for plotting visualization
    vmax_for_genes = {i: np.percentile(vmax_calc[i], 90) for i in vmax_calc}  # Make cutoff 90th percentile of max
    vmin_for_genes = {i: np.percentile(vmin_calc[i], 10) for i in vmin_calc}  # Make cutoff 90th percentile of max

    # Create plots
    fig, ax = plt.subplots(figsize=(len(genes_to_plot) * 4, len(pucks_to_plot) * 4))
    plt.subplots_adjust(hspace=0.5)
    plt_cter = 1

    # Make plots
    for pn in tqdm(pucks_to_plot):
        all_x = all_x_agg[pn]
        all_y = all_y_agg[pn]

        adata = all_adata_filt[pn]  # clean up areas outside tissue
        total_counts_per_cell = all_total_counts[pn]

        for gene_name in genes_to_plot:
            ax = plt.subplot(len(pucks_to_plot), len(genes_to_plot), plt_cter)
            ax.scatter(all_x, all_y, c='black', s=1)  # Plot locations in black

            gene_idx = list(adata.var.index).index(gene_name)
            gene_filt = adata.X[:, gene_idx]
            gene_filt = [i / x * 10000 for i, x in zip(gene_filt, total_counts_per_cell)]
            all_len = len(gene_filt)

            bc_indices = [index for index, val in enumerate(gene_filt) if val > 0]
            gene_filt = [val for index, val in enumerate(gene_filt) if val > 0]
            x = [adata.obs['x_val'][i] for i in bc_indices]
            y = [adata.obs['y_val'][i] for i in bc_indices]

            vmax_calc[gene_name] = vmax_calc[gene_name] + gene_filt

            vmin = vmin_for_genes[gene_name]
            vmax = vmax_for_genes[gene_name]
            thisfig = ax.scatter(x, y, c=gene_filt.toarray(), s=1, cmap='viridis', vmin=vmin, vmax=vmax)
            fig.colorbar(thisfig)
            ax.set_title(f"{gene_name} {len(gene_filt)/all_len:.2%}")

            plt_cter += 1
            ax.set_xlim([0, 5000])
            ax.set_ylim([0, 5000])
            ax.set_aspect('equal')

            scalebar = AnchoredSizeBar(
                ax.transData,
                pixels_needed,
                "",
                "lower right",
                pad=0.1,
                color="black",
                frameon=False,
                size_vertical=1,
                fontproperties=fontprops,
            )
            ax.add_artist(scalebar)

    plt.show()
    plt.savefig(f'{output_directory}{title_name}.png', dpi=300)

def rolling_average(x, y, title):
    # Create a scatter plot of the data points
    fig, ax = plt.subplots() 
    plt.scatter(x, y)
    
    # Calculate the slope (m) and intercept (b) of the line
    m, b = np.polyfit(np.log10(x), y, 1)

    # Generate x values for the line of best fit
    x_fit = np.linspace(min(x), max(x), 10)

    # Calculate corresponding y values using the equation of the line
    y_fit = m * np.log10(x_fit) + b
    
    # Plot the line of best fit
    plt.plot(x_fit, y_fit, color='red', label=f'Line of Best Fit: y = {m:.2f}x + {b:.2f}')

    # Calculate and display Pearson correlation coefficient and p-value
    stat, pval = scipy.stats.pearsonr(np.log10(timepoints), y)
    plt.title(f'R = {round(stat,2)} | p = {round(pval,5)}')
    
    # Set x-axis scale to logarithmic
    ax.set_xscale('log', base=10)
    ax.set_xticks([20, 100, 500])
    ax.get_xaxis().set_major_formatter(mpl.ticker.ScalarFormatter())
    
    # Save the plot as a PDF file
    plt.savefig(f'{output_directory}S6D_{title}_sc_proportion.pdf')

def find_nearest_point_distance(target_point, list_of_points):
    """
    Find the point in a list of points closest to a target point.

    Parameters:
    - target_point (tuple): The target point in the form of (x, y).
    - list_of_points (list of tuples): List of points in the form of [(x1, y1), (x2, y2), ...].

    Returns:
    - nearest_point (tuple): The point in the list closest to the target point.
    - distance_to_nearest (float): The distance between the target point and the nearest point.
    """
    target_point = np.array(target_point)
    
    # Calculate Euclidean distances between the target point and all points in the list
    distances = distance.cdist(target_point, list_of_points, metric='euclidean')
    
    # Find the index of the nearest point
    index_of_nearest = np.argmin(distances)

    # Get the nearest point and its distance to the target point
    nearest_point = list_of_points[index_of_nearest]
    distance_to_nearest = distances[0, index_of_nearest]

    return nearest_point, distance_to_nearest

def calculate_distance_to_boundary(cluster_points, points_of_interest, alpha=0.03, show_plot = False):
    """
    Calculate the distance from each point in a cluster to the boundary of the alpha shape.

    Parameters:
    - cluster_points (list of tuples): List of (x, y) coordinates for the points in the cluster.
    - alpha (float): Alpha parameter for alpha shape. Smaller alpha values will create more detailed shapes.

    Returns:
    - distances (list of floats): List of distances from each point to the boundary.
    - boundary (list of tuples): List of (x, y) coordinates representing the boundary of the alpha shape.
    """

    # Convert the list of tuples to a numpy array for compatibility with alphashape
    points = np.array(cluster_points)
    x,y = zip(*points)
    # Create the alpha shape
    alpha_shape = alphashape(points, alpha=alpha)

    shape_x, shape_y = alpha_shape.exterior.coords.xy
    
    boundary = list(zip(shape_x,shape_y))
    if show_plot:
        plt.figure(figsize=(3,3))
        plt.plot(x, y, 'o', color='black', markersize=6)
        plt.plot(shape_x, shape_y, 'o', color='red', markersize=4)
        plt.xlim([0,5000])
        plt.ylim([0,5000])
        plt.show()
    
    # Calculate the distance from each point to the boundary
    distances = [find_nearest_point_distance([point], boundary) for point in points_of_interest]

    return alpha_shape, distances, boundary

# Plot cell_types
def plot_cell_types(adata, rctd_results, cell_type_list, title, title_save = None, colors=None, point_size=0.5):
    """
    Plot cell types on a spatial map.

    Parameters:
    - adata: AnnData object containing spatial information.
    - rctd_results: DataFrame containing RCTD results.
    - cell_type_list: List of cell types to plot.
    - title: Title of the plot.
    - colors: List of colors for each cell type.
    - point_size: Size of the plotted points.
    """
    if colors is None:
        colors = sns.color_palette('husl', len(cell_type_list))

    fig, ax = plt.subplots(figsize=(5, 5))
    plt.xlim([0, 5000])
    plt.ylim([0, 5000])

    try:
        loc_list = zip(adata.obs.x_val, adata.obs.y_val)
    except:
        loc_list = zip(adata.obs.x, adata.obs.y)
    
    adata.obs['barcode'] = [i.split('_')[2] for i in adata.obs.index]
    bc_to_loc = {i: j for i, j in zip(adata.obs.barcode, loc_list)}

    # Filter data for cortex and medulla regions
    adata_filt = adata[adata.obs.cortex_medulla.isin(['cortex', 'medulla'])]
    
    try:
        all_x, all_y = [adata_filt.obs.x_val, adata_filt.obs.y_val]
    except:
        all_x, all_y = [adata_filt.obs.x, adata_filt.obs.y]
    plt.scatter(all_x, all_y, alpha=0.025, color='gray', s=1)

    # Plot each cell type
    for idx in range(len(cell_type_list)):
        ct = cell_type_list[idx]
        bcs = rctd_results[rctd_results['first_type'] == ct]['Unnamed: 0']
        locs = [bc_to_loc[i] for i in bcs]
        x, y = zip(*locs)
        plt.scatter(x, y, s=point_size, c=colors[idx], label=ct)

    # Add scale bar
    scalebar = AnchoredSizeBar(
        ax.transData,
        pixels_needed,
        "",
        "lower right",
        pad=0.1,
        color="black",
        frameon=False,
        size_vertical=1,
        fontproperties=fontprops,
    )
    ax.add_artist(scalebar)
    
    plt.title(title)
    plt.legend() 
    if title_save!=None:
        plt.savefig(f'{output_directory}{title_save}_{cell_type_list}_{title}.pdf')
    plt.show()

# Establish time point groupings
def create_bins_dict(time_points,num_bins = 10, correct_timepoints = True): # default is deciles # correct timepoints makes sure the same time point is in the same decile
    # Sort the time points in ascending order
    sorted_time_points = sorted(time_points)
    
    # Calculate the indices for deciles
    decile_indices = [int(len(sorted_time_points) * i / num_bins) for i in range(num_bins)]
    
    # Create the dictionary of deciles
    deciles_dict = {}
    
    for i in range(num_bins):
        decile_key = f'Decile {i+1}'
        if i < num_bins-1:
            decile_values = sorted_time_points[decile_indices[i]:decile_indices[i+1]]
        else:
            decile_values = sorted_time_points[decile_indices[i]:]
        deciles_dict[decile_key] = decile_values
    # Add the last decile separately
    deciles_dict[f'Decile {num_bins}'] = sorted_time_points[decile_indices[-1]:]
    
    if correct_timepoints:
        deciles_dict_corrected = {}
        for i in range(num_bins):
            decile_key = f'Decile {i+1}'
            decile_key_old = f'Decile {i}'
            if i == 0:
                deciles_dict_corrected[decile_key] = deciles_dict[decile_key]
                continue
            else:
                new = deciles_dict[decile_key]
                old = deciles_dict_corrected[decile_key_old]
                old_updated = old + [i for i in new if i in old] 
                new_updated =  [i for i in new if i not in old]
                
                deciles_dict_corrected[decile_key_old] = old_updated
                deciles_dict_corrected[decile_key] = new_updated
        deciles_dict = deciles_dict_corrected
            
            
    return deciles_dict

def subset_adata(adata, age, location):
    """
    Filter the input AnnData object based on age and location.

    Parameters:
        adata (AnnData): The input AnnData object.
        age (str): Age category ('young' or 'old').
        location (str): Location category ('cortex' or 'medulla').

    Returns:
        adata_filt (AnnData): Filtered AnnData object based on age and location.
    """
    # Filter AnnData based on age
    adata_filt = adata[adata.obs['Age'].isin(age)]
    # Further filter based on location
    adata_filt = adata_filt[adata_filt.obs['cortex_medulla'] == location]
    return adata_filt

def young_old_props(adata):
    """
    Calculate cell type proportions for young and old samples.

    Parameters:
        adata (AnnData): The input AnnData object.

    Returns:
        young (list): List of dictionaries containing cell type proportions for young samples.
        old (list): List of dictionaries containing cell type proportions for old samples.
    """
    young = []
    old = []
    for age in tqdm(set(adata.obs['Age'])):
        # Subset AnnData for the specific age
        adata_filt_age = adata[adata.obs['Age'] == age]
        # Calculate cell type proportions
        cter = Counter(adata_filt_age.obs['Cell_type'])
        cter_norm = {i: cter[i] / sum(cter.values()) for i in cter}
        # Append proportions to young or old based on age category
        if age in young_ages:
            young.append(cter_norm)
        elif age in old_ages:
            old.append(cter_norm)
        else:
            print('Unexpected age category')
    return young, old

def cohens_d(c0, c1):
    """
    Calculate Cohen's d effect size.

    Parameters:
        c0 (list): Values for group 0.
        c1 (list): Values for group 1.

    Returns:
        cohens_d (float): Cohen's d effect size.
    """
    cohens_d = (mean(c0) - mean(c1)) / (sqrt((stdev(c0) ** 2 + stdev(c1) ** 2) / 2))
    return cohens_d

def find_cohens_d_by_ct(adata, young, old):
    """
    Find Cohen's d effect size by cell type.

    Parameters:
        adata (AnnData): The input AnnData object.
        young (list): List of dictionaries containing cell type proportions for young samples.
        old (list): List of dictionaries containing cell type proportions for old samples.

    Returns:
        ct (list): List of cell types sorted by Cohen's d effect size.
        vals (list): List of Cohen's d effect sizes sorted by magnitude.
        pvals (dict): Dictionary of p-values for t-tests between young and old samples for each cell type.
    """
    cohens_d_by_cell_type = {}
    pvals = {}
    for cell_type in tqdm(set(adata_filt.obs['Cell_type'])):
        # Get proportions for young and old samples for the current cell type
        young_cell_type_prop = [i[cell_type] for i in young if cell_type in i]
        old_cell_type_prop = [i[cell_type] for i in old if cell_type in i]

        # Calculate Cohen's d effect size
        cd = cohens_d(old_cell_type_prop, young_cell_type_prop)
        cohens_d_by_cell_type[cell_type] = cd

        # Perform t-test and store p-value
        stat, pval = scipy.stats.ttest_ind(young_cell_type_prop, old_cell_type_prop)
        pvals[cell_type] = pval

    # Sort cell types by Cohen's d effect size
    sorted_cohens_d_by_cell_type = sorted(cohens_d_by_cell_type.items(), key=lambda x: x[1], reverse=True)
    ct = [i[0] for i in sorted_cohens_d_by_cell_type]
    vals = [i[1] for i in sorted_cohens_d_by_cell_type]
    return ct, vals, pvals

with open(f'{directory}Thymus_data_TCR_locs_diver.h5',"rb") as handle:
    adata_diver = sc.read_h5ad(handle) 

with open(f'{directory}thymuscombined_xy.h5ad', "rb") as handle:
    adata_combined = sc.read_h5ad(handle)

/opt/conda/lib/python3.9/site-packages/numba/core/decorators.py:246: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


# Figure S3

## S3D

In [ ]:
adata = copy.deepcopy(adata_combined)

In [ ]:
region_type = 'medulla'

# Initialize dictionaries to store data
all_cell_type_pairs = {}  # Dictionary to store z-scores for cell type pairs
arr_all = {}  # Dictionary to store z-score DataFrames for each timepoint

# Iterate over each puck name and load corresponding data
for puck_name in tqdm(puck_names):
    # Skip specific pucks with insufficient cells in the medulla region
    if region_type == 'medulla' and puck_name in {
        '2022-11-23_Puck_221024_34',  # Not enough cells, 2 days
        '2023-01-24_Puck_221103_09',  # Not enough cells, 4 days
        '2023-01-24_Puck_221103_12',  # Not enough cells, 4 days
    }:
        continue
    
    # Define the file path for loading the data
    file_path = f'{directory}permutation_testing/zscore_{puck_name}_pd_{region_type}.pkl'
    
    try:
        # Load the data from the file
        with open(file_path, 'rb') as f:
            z_score_df = pk.load(f)
    except FileNotFoundError:
        # Handle missing data files gracefully
        print(f'File missing: {puck_name}')
        continue

# Process and store data for each puck
for puck_name in puck_names:
    # Get the corresponding timepoint
    timepoint = puck_to_time[puck_name]

    # Define the file path
    file_path = f'{directory}permutation_testing/zscore_{puck_name}_pd_{region_type}.pkl'

    try:
        # Load data from the file
        with open(file_path, 'rb') as f:
            df = pk.load(f)
    except FileNotFoundError:
        print(f'{timepoint} {puck_name} is missing')
        continue

    # Store the loaded DataFrame in the dictionary
    arr_all[timepoint] = df

    # Extract cell type labels from DataFrame
    cell_types_col = df.columns  # Column cell types
    cell_types_row = df.index    # Row cell types
    z_scores = np.array(df)  # Convert DataFrame to a NumPy array for easier indexing

    # Iterate over each cell type pair
    for i, c1 in enumerate(cell_types_col):
        for j, c2 in enumerate(cell_types_row):
            # Store the z-score for this cell type pair at the given timepoint
            if (c1, c2) not in all_cell_type_pairs:
                all_cell_type_pairs[(c1, c2)] = []
            all_cell_type_pairs[(c1, c2)].append((timepoint, z_scores[i, j]))  # Append (timepoint, z-score)


In [ ]:
# Initialize an empty dictionary to store line plots for each cell type pair
line_plots = {}

# Iterate over all cell type pairs
for i in all_cell_type_pairs:
    num_pairs = len(all_cell_type_pairs[i])
    line_plots[i] = all_cell_type_pairs[i]

# Boolean indicating whether to correct p-values
corr_pval = True

# Initial color for plotting
color = 'gray'

# Initialize a list to store p-values for correction
pval_recorder_for_corr = []

# Counter for the subplots
plt_cter = 0

# Iterate over each cell type pair for plotting
for i in line_plots:
    if len(line_plots[i]) <= 1:
        continue
    # Filter out NaN values from the line plots
    line_plots_filt = [j for j in line_plots[i] if ~np.isnan(j[1])]
    y_vals = [j[1] for j in line_plots_filt]
    x_vals = [tpt_to_day[j[0]] for j in line_plots_filt]
    x_vals = [i+1 for i in x_vals]
    # Sort x and y values
    x_vals, y_vals = zip(*sorted([(i,j) for i,j in zip(x_vals,y_vals)]))
    x_vals, y_vals = zip(*sorted([(i,j) for i,j in zip(x_vals,y_vals) if np.isfinite(j)]))
    
    if len(x_vals) < 5:
        continue
    # Calculate Pearson correlation coefficient and p-value
    corr, pval = scipy.stats.pearsonr(np.log10(x_vals), y_vals)

    # Store p-value for correction
    pval_recorder_for_corr.append(pval)

# Perform multiple testing correction on p-values
corr_list = stats.multitest.multipletests(pval_recorder_for_corr, method='fdr_bh')
corr_dict = {i: j for i, j in zip(pval_recorder_for_corr, corr_list[1])}

# For plotting
dotplot_sig_interactions = []
dotplot_sig_corr = []
dotplot_sig_pval = []

# Create subplots for individual interactions

if region_type == 'cortex':
    fig, axs = plt.subplots(nrows=35, ncols=5, figsize=(16, 7*12))
elif region_type == 'medulla':
    fig, axs = plt.subplots(nrows=5, ncols=5, figsize=(16, 12))
    
plt.subplots_adjust(hspace=0.5)

# Reset the counter for subplots
plt_cter = 0

# Iterate over each cell type pair for plotting
for i in tqdm(line_plots):
    if len(line_plots[i]) <= 1:
        continue
    line_plots_filt = [j for j in line_plots[i] if ~np.isnan(j[1])]
    y_vals = [j[1] for j in line_plots[i]]
    x_vals = [tpt_to_day[j[0]] for j in line_plots[i]]
    x_vals = [i+1 for i in x_vals]
    x_vals, y_vals = zip(*sorted([(i,j) for i,j in zip(x_vals,y_vals) if np.isfinite(j)]))
    if len(x_vals) < 5:
        continue
    # Calculate Pearson correlation coefficient and p-value
    corr, pval = scipy.stats.pearsonr(np.log10(x_vals), y_vals)
    if corr_pval:
        # Correct p-value if needed
        pval = corr_dict[pval]
    dotplot_sig_interactions.append(i)
    dotplot_sig_corr.append(corr)
    dotplot_sig_pval.append(pval)
    
    # Plot significant interactions
    if (pval < 0.05):
        y_vals_df = pd.DataFrame.from_dict({'y': y_vals})
        y_vals_line = y_vals_df['y'].rolling(window=5, min_periods=1, center=True).mean()
        alpha_val = 1
        print(f'{i[0]} and {i[1]}')
        ax_val = axs.ravel()[plt_cter]
        plt_cter += 1
        ax_val.scatter(x_vals, y_vals, c=color, alpha=alpha_val, s=5)
        ax_val.plot(x_vals, y_vals_line, c='black', alpha=alpha_val)
        
        ax_val.set_title(str(i)+' \n pval: '+str(round(pval, 8)) + ' corr: '+str(round(corr, 2)))
        ax_val.set_xscale('log', base=10)
        ax_val.set_xticks([1, 5, 10, 100, 500])
        ax_val.get_xaxis().set_major_formatter(ScalarFormatter())
        
    else:
        color = 'gray'
        alpha_val = 0.01
        continue
        
# Set the label for x-axis
plt.xlabel('timepoints, log+1')


In [ ]:
# Extract unique cell types from dotplot_sig_interactions
all_ct = list(set([i[0] for i in dotplot_sig_interactions] + [i[1][:-2] for i in dotplot_sig_interactions]))

# Remove specific cell types based on region type
if region_type == 'medulla':
    all_ct = [i for i in all_ct if 'cTEC' not in i]
elif region_type == 'cortex':
    all_ct = [i for i in all_ct if 'mTEC' not in i]

# Initialize matrices for heatmap
heatmap_mat = np.zeros([len(all_ct),len(all_ct)])
heatmap_pval = np.zeros([len(all_ct),len(all_ct)])

# Loop over each interaction
for idx in range(len(dotplot_sig_interactions)):
    sig_int = dotplot_sig_interactions[idx]
    corr = dotplot_sig_corr[idx]
    pval = dotplot_sig_pval[idx]

    # Skip interactions involving specific cell types based on region type
    ct1 = sig_int[0]
    ct2 = sig_int[1]
    if region_type == 'medulla':
        if 'cTEC' in ct1 or 'cTEC' in ct2:
            continue
    elif region_type == 'cortex':
        if 'mTEC' in ct1 or 'mTEC' in ct2:
            continue
    
    # Get indices of cell types in the heatmap matrices
    ct1_idx = all_ct.index(ct1)
    ct2_idx = all_ct.index(ct2[:-2])
    
    # Populate heatmap matrices
    heatmap_mat[ct1_idx][ct2_idx] = corr
    heatmap_pval[ct1_idx][ct2_idx] = pval

# Create DataFrame for heatmap data and remove rows/columns with all zeros
heatmap_df = pd.DataFrame(heatmap_mat,index=all_ct,columns=all_ct)
heatmap_df = heatmap_df.loc[~(heatmap_df==0).all(axis=1)]
heatmap_df = heatmap_df.loc[:, (heatmap_df != 0).any(axis=0)]

# Create DataFrame for p-values and remove rows/columns with all zeros
heatmap_pv = pd.DataFrame(heatmap_pval,index=all_ct,columns=all_ct)
heatmap_pv = heatmap_pv.loc[~(heatmap_pv==0).all(axis=1)]
heatmap_pv = heatmap_pv.loc[:, (heatmap_pv != 0).any(axis=0)]

# Create clustermap to reorder the data frame
cm = sns.color_palette("vlag", as_cmap=True)
fig = sns.clustermap(heatmap_df,cmap=cm,xticklabels=True,yticklabels=True,
                     figsize=(15,10),row_cluster=True,col_cluster=True)
fig.savefig('cortex_interaction_heatmap.pdf')

# Reorder data frame based on clustermap
ind = fig.dendrogram_col.reordered_ind
set_order = [heatmap_df.index[i] for i in ind]
if region_type == 'medulla':
    set_order = ['DC2', 'Mac', 'B', 'CD4+T', 'CD8+T', 'Ery', 'Endo', 'Fb', 'DC1', 'Treg', 'alpha_beta_T(entry)', 'pDC', 'Epi_unknown', 'VSMC', 'NK', 'IELpA', 'HSC', 'TEC_early', 'IELpB_NKT', 'Mono', 'DN(Q)', 'DP(Q)', 'mTEC', 'gdT', 'DN(P)', 'DP(P)']
elif region_type == 'cortex':
    set_order = ['DC2', 'Mac', 'B', 'CD4+T', 'CD8+T', 'Ery', 'Endo', 'Fb', 'DC1', 'Treg', 'alpha_beta_T(entry)', 'pDC', 'Epi_unknown', 'VSMC', 'NK', 'IELpA', 'HSC', 'TEC_early', 'IELpB_NKT', 'Mono', 'DN(Q)', 'DP(Q)', 'cTEC', 'gdT', 'DN(P)', 'DP(P)']
heatmap_df = heatmap_df[set_order]
heatmap_df = heatmap_df.reindex(index = set_order)

heatmap_pv = heatmap_pv[set_order]
heatmap_pv = heatmap_pv.reindex(index = set_order)

# Create scattered heat map
M = 26
N = 26
xlabels = heatmap_df.columns
ylabels = heatmap_df.index

x,y = np.meshgrid(np.arange(M), np.arange(N))
s = -np.log10(heatmap_pv.values) # Size
c = heatmap_df.values # Color

fig,ax = plt.subplots(figsize=(10,10))

R = (s+2)/(s.max()+2)/2
circles = [plt.Circle((j,i), radius=r) for r, j, i in zip(R.flat, x.flat, y.flat)]
col = PatchCollection(circles, array=c.flatten(), cmap="vlag")
ax.add_collection(col)

ax.set(xticks=np.arange(M), yticks=np.arange(N),
       xticklabels=xlabels, yticklabels=ylabels)
ax.set_xticks(np.arange(M+1)-0.5, minor=True)
ax.set_yticks(np.arange(N+1)-0.5, minor=True)
ax.grid(which='minor')
plt.xticks(rotation = 90)

plt.savefig(f'fig2b_cb_{region_type}.pdf')
plt.show()

# Create scattered heat map with colorbar
fig,ax = plt.subplots(figsize=(10,10))

R = (s+2)/(s.max()+2)/2
circles = [plt.Circle((j,i), radius=r) for r, j, i in zip(R.flat, x.flat, y.flat)]
col = PatchCollection(circles, array=c.flatten(), cmap="vlag")
ax.add_collection(col)

ax.set(xticks=np.arange(M), yticks=np.arange(N),
       xticklabels=xlabels, yticklabels=ylabels)
ax.set_xticks(np.arange(M+1)-0.5, minor=True)
ax.set_yticks(np.arange(N+1)-0.5, minor=True)
ax.grid(which='minor')
plt.xticks(rotation = 90)

fig.colorbar(col)

plt.show()
plt.savefig(f'{output_directory}S3D.pdf')

## S3E

In [ ]:
adata = copy.deepcopy(adata_combined)

In [ ]:
region_types = ['cortex','medulla']

for region_type in region_types:
    print(region_type)
    print()
    # Initialize dictionaries to store data
    all_cell_type_pairs = {}  # Dictionary to store z-scores for cell type pairs
    arr_all = {}  # Dictionary to store z-score DataFrames for each timepoint
    
    # Iterate over each puck name and load corresponding data
    for puck_name in tqdm(puck_names):
        # Skip specific pucks with insufficient cells in the medulla region
        if region_type == 'medulla' and puck_name in {
            '2022-11-23_Puck_221024_34',  # Not enough cells, 2 days
            '2023-01-24_Puck_221103_09',  # Not enough cells, 4 days
            '2023-01-24_Puck_221103_12',  # Not enough cells, 4 days
        }:
            continue
        
        # Define the file path for loading the data
        file_path = f'{directory}permutation_testing/zscore_{puck_name}_pd_{region_type}.pkl'
        
        try:
            # Load the data from the file
            with open(file_path, 'rb') as f:
                z_score_df = pk.load(f)
        except FileNotFoundError:
            # Handle missing data files gracefully
            print(f'File missing: {puck_name}')
            continue
    
    # Process and store data for each puck
    for puck_name in puck_names:
        # Get the corresponding timepoint
        timepoint = puck_to_time[puck_name]
    
        # Define the file path
        file_path = f'{directory}permutation_testing/zscore_{puck_name}_pd_{region_type}.pkl'
    
        try:
            # Load data from the file
            with open(file_path, 'rb') as f:
                df = pk.load(f)
        except FileNotFoundError:
            print(f'{timepoint} {puck_name} is missing')
            continue
    
        # Store the loaded DataFrame in the dictionary
        arr_all[timepoint] = df
    
        # Extract cell type labels from DataFrame
        cell_types_col = df.columns  # Column cell types
        cell_types_row = df.index    # Row cell types
        z_scores = np.array(df)  # Convert DataFrame to a NumPy array for easier indexing
    
        # Iterate over each cell type pair
        for i, c1 in enumerate(cell_types_col):
            for j, c2 in enumerate(cell_types_row):
                # Store the z-score for this cell type pair at the given timepoint
                if (c1, c2) not in all_cell_type_pairs:
                    all_cell_type_pairs[(c1, c2)] = []
                all_cell_type_pairs[(c1, c2)].append((timepoint, z_scores[i, j]))  # Append (timepoint, z-score)
    # Initialize an empty dictionary to store line plots for each cell type pair
    line_plots = {}
    
    # Iterate over all cell type pairs
    for i in all_cell_type_pairs:
        num_pairs = len(all_cell_type_pairs[i])
        line_plots[i] = all_cell_type_pairs[i]
    
    # Boolean indicating whether to correct p-values
    corr_pval = True
    
    # Initial color for plotting
    color = 'gray'
    
    # Initialize a list to store p-values for correction
    pval_recorder_for_corr = []
    
    # Counter for the subplots
    plt_cter = 0
    
    # Iterate over each cell type pair for plotting
    for i in line_plots:
        if len(line_plots[i]) <= 1:
            continue
        # Filter out NaN values from the line plots
        line_plots_filt = [j for j in line_plots[i] if ~np.isnan(j[1])]
        y_vals = [j[1] for j in line_plots_filt]
        x_vals = [tpt_to_day[j[0]] for j in line_plots_filt]
        x_vals = [i+1 for i in x_vals]
        # Sort x and y values
        x_vals, y_vals = zip(*sorted([(i,j) for i,j in zip(x_vals,y_vals)]))
        x_vals, y_vals = zip(*sorted([(i,j) for i,j in zip(x_vals,y_vals) if np.isfinite(j)]))
        
        if len(x_vals) < 5:
            continue
        # Calculate Pearson correlation coefficient and p-value
        corr, pval = scipy.stats.pearsonr(np.log10(x_vals), y_vals)
    
        # Store p-value for correction
        pval_recorder_for_corr.append(pval)
    
    # Perform multiple testing correction on p-values
    corr_list = stats.multitest.multipletests(pval_recorder_for_corr, method='fdr_bh')
    corr_dict = {i: j for i, j in zip(pval_recorder_for_corr, corr_list[1])}
    
    # For plotting
    dotplot_sig_interactions = []
    dotplot_sig_corr = []
    dotplot_sig_pval = []
    
    # Create subplots for individual interactions
    
    if region_type == 'cortex':
        fig, axs = plt.subplots(nrows=35, ncols=5, figsize=(16, 7*12))
    elif region_type == 'medulla':
        fig, axs = plt.subplots(nrows=5, ncols=5, figsize=(16, 12))
        
    plt.subplots_adjust(hspace=0.5)
    
    # Reset the counter for subplots
    plt_cter = 0
    
    # Iterate over each cell type pair for plotting
    for i in tqdm(line_plots):
        if len(line_plots[i]) <= 1:
            continue
        line_plots_filt = [j for j in line_plots[i] if ~np.isnan(j[1])]
        y_vals = [j[1] for j in line_plots[i]]
        x_vals = [tpt_to_day[j[0]] for j in line_plots[i]]
        x_vals = [i+1 for i in x_vals]
        x_vals, y_vals = zip(*sorted([(i,j) for i,j in zip(x_vals,y_vals) if np.isfinite(j)]))
        if len(x_vals) < 5:
            continue
        # Calculate Pearson correlation coefficient and p-value
        corr, pval = scipy.stats.pearsonr(np.log10(x_vals), y_vals)
        if corr_pval:
            # Correct p-value if needed
            pval = corr_dict[pval]
        dotplot_sig_interactions.append(i)
        dotplot_sig_corr.append(corr)
        dotplot_sig_pval.append(pval)
        
        # Plot significant interactions
        if (pval < 0.05):
            y_vals_df = pd.DataFrame.from_dict({'y': y_vals})
            y_vals_line = y_vals_df['y'].rolling(window=5, min_periods=1, center=True).mean()
            alpha_val = 1
            print(f'{i[0]} and {i[1]}')
            ax_val = axs.ravel()[plt_cter]
            plt_cter += 1
            ax_val.scatter(x_vals, y_vals, c=color, alpha=alpha_val, s=5)
            ax_val.plot(x_vals, y_vals_line, c='black', alpha=alpha_val)
            
            ax_val.set_title(str(i)+' \n pval: '+str(round(pval, 8)) + ' corr: '+str(round(corr, 2)))
            ax_val.set_xscale('log', base=10)
            ax_val.set_xticks([1, 5, 10, 100, 500])
            ax_val.get_xaxis().set_major_formatter(ScalarFormatter())
            
        else:
            color = 'gray'
            alpha_val = 0.01
            continue
            
    # Set the label for x-axis
    plt.xlabel('timepoints, log+1')
    plt.savefig(f'{output_directory}S3E_{region_type}.pdf')
    plt.show()

